In [ ]:
!pip install faiss-gpu
!pip install pytorch_metric_learning
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 77.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 38.0 MB/s eta 0:00:00


In [ ]:
import faiss
from tqdm import tqdm
import numpy as np
import pickle as pkl
from transformers import AutoModel, AutoTokenizer
from sklearn.decomposition import PCA
import random

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/sapBERT-DUT-cambridge')
sapBERT_epochs = 3
finetune_epochs = 3
finetune_dataset = 'nl_wiki_bel_all'
model_directory_path = f'/content/drive/MyDrive/sapBERT-DUT-cambridge/results/medRoBERTa_sapBERT/{sapBERT_epochs}_epoch/ft_{finetune_epochs}_epoch'

In [ ]:
cuis = []
concepts = []
with open('/content/drive/MyDrive/sapBERT-DUT-cambridge/ontologies/umls-dutch_v1.11_with_drugs_filtered-categories.csv') as f:
    # skip header
    next(f)
    for line in f:
        cui = line.split(',')[0]
        concept = line.split(',')[1]
        cuis.append(cui)
        concepts.append(concept)

In [ ]:
from src.model_wrapper import (
    Model_Wrapper
)

In [ ]:
model_wrapper = Model_Wrapper().load_model(
        path=f'{model_directory_path}',
        max_length=25,
        use_cuda=True,
)

In [ ]:
def reduce_embedding_dim(embeddings):
    pca_train_set = random.sample(list(embeddings), k=int(len(embeddings) * 0.5))
    pca = PCA(n_components=256)
    pca.fit(pca_train_set)
    pkl.dump(pca, open(f'{model_directory_path}/pca', 'wb'))
    reduced_embeddings = np.array(pca.transform(embeddings), dtype=np.float32)
    return reduced_embeddings

In [ ]:
tokenizer = model_wrapper.get_dense_tokenizer()
model = model_wrapper.get_dense_encoder()

# tokenizer = AutoTokenizer.from_pretrained('CLTL/MedRoBERTa.nl')
# model = AutoModel.from_pretrained('CLTL/MedRoBERTa.nl').to('cuda')

bs = 128
all_reps = []
for i in tqdm(np.arange(0, len(concepts), bs)):
    toks = tokenizer.batch_encode_plus(concepts[i:i+bs],
                                       padding="max_length",
                                       max_length=25,
                                       truncation=True,
                                       return_tensors="pt")
    toks_cuda = {}
    for k,v in toks.items():
       toks_cuda[k] = v.cuda()
    output = model(**toks_cuda)

    # output = model(**toks)
    cls_rep = output[0][:,0,:]

    all_reps.append(cls_rep.cpu().detach().numpy())
all_reps_emb = np.concatenate(all_reps, axis=0)

pkl.dump(all_reps_emb, open(f'{model_directory_path}/all_reps_emb', "wb"))


100%|██████████| 5880/5880 [16:57<00:00,  5.78it/s]


In [ ]:
# Build faiss index from embeddings
logging.info(f"Training index with embedding dim size {cfg.dims} using {faiss.get_num_gpus()} gpus")
embeddings = pkl.load(open(f'{model_directory_path}/all_reps_emb', 'rb'))
embeddings = reduce_embedding_dim(embeddings)

quantizer = faiss.IndexFlatL2(256)
index = faiss.IndexIVFFlat(quantizer, 256, 300)
index = faiss.index_cpu_to_all_gpus(index)
index.train(embeddings)


logging.info("Adding dataset embeddings to index")
for i in tqdm(range(0, embeddings.shape[0], 1000)):
    index.add(embeddings[i : i + 1000])

logging.info("Saving index")
faiss.write_index(faiss.index_gpu_to_cpu(index), f'{model_directory_path}/index')

logging.info("Index built and saved")

100%|██████████| 753/753 [00:00<00:00, 965.76it/s] 
